In [134]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot =True)
x =tf.placeholder(tf.float32, [None,784])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


此处有两个部分，**占位符**和**变量**。他们都代表Tensor。tensor的概念为非具体数值，而是部分希望计算的节点。  
**占位符**， 它的值由由用户自行传递给TensorFlow。通常存储样本的数据和标签。比如x =tf.placeholder(tf.float32, [None,784])存储训练图片数据的占位符。形状为[None,784]，可以传递任意张训练图片给这个占位符，每张图片用一个784维的向量表示。占位符并没有初始值，它只会分配必要的内存。  
**变量**，代表任何时候可以改变的值，每次计算后的值会被保存下来。  

In [135]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W)+b)
y_ = tf.placeholder(tf.float32, [None,10])

tf.reduce_mean 函数用于计算张量tensor沿着指定的数轴（tensor的某一维度）上的的平均值，主要用作降维或者计算tensor（图像）的平均值。

# Cross_entropy loss func

In [136]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y)))#下面有reduce_mean和reduce_sum相关的例子

#此处是关于reduce_mean()的参考代码
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
 
x = [[1,2,3],
      [1,2,3]]
 
xx = tf.cast(x,tf.float32)
 
mean_all = tf.reduce_mean(xx, keepdims=False)#keep_dims
mean_0 = tf.reduce_mean(xx, axis=0, keepdims=False)# axis=0,表示列上面求均值，axis=1， 表示行上面求均值 
mean_1 = tf.reduce_mean(xx, axis=1, keepdims=False)
 
 
with tf.Session() as sess:
    m_a,m_0,m_1 = sess.run([mean_all, mean_0, mean_1])
 
print(m_a)    # output: 2.0
print(m_0)    # output: [ 1.  2.  3.]
print(m_1)    #output:  [ 2.  2.]

类似函数还有:  
tf.reduce_sum ：计算tensor指定轴方向上的所有元素的累加和;  
tf.reduce_max  :  计算tensor指定轴方向上的各个元素的最大值;  
tf.reduce_all :  计算tensor指定轴方向上的各个元素的逻辑和（and运算）;  
tf.reduce_any:  计算tensor指定轴方向上的各个元素的逻辑或（or运算）;  

# Gradient descent 

In [137]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

此处两个自变量，W和b，那么利用梯度下降发对W和b进行更新。update w and b value. 

#### 在优化前，需要进行初始化，这里叫做创建会话，并进行初始化。

In [138]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()#初始化所有变量
print('start training...')

start training...


# Optimization start

In [139]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    #get 100 training data 
    #shape batch_xs(100,784), batch_ys(100,10), batch_xs and batch_ys are placeholders of xs and y_
    #运行会话，参数：train_step, batches for xs and ys
    sess.run(train_step, feed_dict = {x:batch_xs, y_: batch_ys})
correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(y_,1))#取出行中最大的下标
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#具体作用，参考下面带有数值的具体代码
print(sess.run(accuracy, feed_dict ={x:mnist.test.images, y_:mnist.test.labels}))

0.9011


tf.equal例子 https://blog.csdn.net/ustbbsy/article/details/79564529

In [143]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
''''x = np.ones((4,9))
x = 0.01*x[:,:]
x[0,0]=0.91
x[1,0]=0.91
x[2,0]=0.91
x[3,0]=0.91
print(np.array(x))
print(type(x))'''
x  = [[1,3,4,5,6]]
x1 = [[1,3,4,3,2]]
print(x)
with tf.Session()  as sess:
    print(sess.run(tf.equal(x,x1)))

[[1, 3, 4, 5, 6]]
[[ True  True  True False False]]


In [141]:
import tensorflow.compat.v1 as tf
import numpy as np
 
A = [[1,3,4,5,6]]
B = [[1,3,4,3,2]]
 
with tf.Session() as sess:
    print(sess.run(tf.equal(A, B)))

[[ True  True  True False False]]


In [142]:
import tensorflow.compat.v1 as tf
a = [[1,2,3],[4,5,6]]
b = [[1,0,3],[1,5,1]]
with tf.Session() as sess:
    print(sess.run(tf.equal(a,b)))

[[ True False  True]
 [False  True False]]


比如上述代码，得到false或者true，true代表预测正确，相对的false就是预测错误，那么就可以将其换成对应的1和0，从而算出来[1,0,0,1], 所以其准确率是50%